# Is het verdamping?

In dit experiment is er een verwarmingselement in een met water gevulde maatbeker gestopt. Elke minuut is de temperatuur van het water gemeten. Deze metingen zijn opgeslagen in [tempmetingen.csv](tempmetingen.csv).

Verder is gegeven dat: 
$m_{maatbeker} = 820.8 \mathrm{gr}$  
$m_{maatbeker + water, start} = 1292.9 \mathrm{gr}$  
$m_{maatbeker + water, eind} = 1274.9 \mathrm{gr}$

Het moge duidelijk zijn dat er water is 'verdwenen'. Ook is de eindtemperatuur van het water niet gelijk aan een proces waarbij verdamping en eventuele warmteverliezen niet meegenomen worden. 

```{exercise}
Onderzoek het proces door:
1. De plot van de metingen te maken, inclusief een verwachte trendlijn voor de temperatuur van het water als al de toegevoerde warmte gaat zitten in het water.
2. Bepaal of de totale hoeveelheid toegevoegde energie correspondeert met de opwarming en verdamping van het water, geef daarbij de aannames die je doet.
3. Geef aanbevelingen voor een verbetering van het experiment om (nog) beter het thermodynamische proces te kunnen onderzoeken en beschrijven.
```

In [7]:
import numpy as np
import matplotlib.pyplot as plt
import glob

# GEGEVEN WAARDEN
m_beker = 820.8
m_beker_water_start = 1292.9
m_beker_water_eind = 1274.9

m_water_start = m_beker_water_start - m_beker
m_water_eind = m_beker_water_eind - m_beker
m_water_verdampt = m_water_start - m_water_eind

Data = np.loadtxt('tempmetingen.csv')



FileNotFoundError: tempmetingen.csv not found.

In [ ]:
# VRAAG 1: TEMPERATUUR PLOT
print("\n" + "="*70)
print("❶ TEMPERATUURVERLOOP")
print("="*70)

if T is not None:
    tijd = np.arange(len(T))
    T_start = T[0]
    T_eind = T[-1]
    delta_T = T_eind - T_start
    
    print(f"\n{T_start:.1f}°C → {T_eind:.1f}°C (Δ{delta_T:.1f}°C in {len(T)-1} min)")
    
    # Lineaire trend
    z = np.polyfit(tijd, T, 1)
    T_fit = np.polyval(z, tijd)
    
    # Plot
    plt.figure(figsize=(11, 5))
    plt.scatter(tijd, T, color='red', s=50, alpha=0.6, zorder=3)
    plt.plot(tijd, T_fit, 'b--', lw=2)
    plt.xlabel('Tijd (min)', fontweight='bold')
    plt.ylabel('Temperatuur (°C)', fontweight='bold')
    plt.grid(alpha=0.3)
    plt.title('Temperatuurstijging', fontweight='bold')
    plt.tight_layout()
    plt.show()
else:
    delta_T = 70
    print(f"Geen data - gebruik Δ{delta_T}°C als voorbeeld")

# VRAAG 2: ENERGIEBALANS
print("\n" + "="*70)
print("❷ ENERGIEBALANS")
print("="*70)

c_water = 4186
c_beker = 385
L_vap = 2.4e6

# Opwarming
Q_w = (m_water_start/1000) * c_water * delta_T
Q_b = (m_beker/1000) * c_beker * delta_T
Q_opw = Q_w + Q_b

print(f"\n1. Opwarming:")
print(f"   Water: {Q_w/1e3:.1f} kJ")
print(f"   Beker: {Q_b/1e3:.1f} kJ")
print(f"   Subtotaal: {Q_opw/1e3:.1f} kJ")

# Verdamping
Q_verd = (m_water_verdampt/1000) * L_vap
print(f"\n2. Verdamping:")
print(f"   m = {m_water_verdampt:.1f} g")
print(f"   Q = {Q_verd/1e3:.1f} kJ")

# Totaal
Q_tot = Q_opw + Q_verd
t_min = (len(T)-1) if T is not None else 30
P_avg = Q_tot / (t_min * 60)

print(f"\n📊 Totaal:")
print(f"   E = {Q_tot/1e3:.1f} kJ")
print(f"   t = {t_min} min")
print(f"   P = {P_avg:.0f} W")

# VRAAG 3: AANBEVELINGEN
print("\n" + "="*70)
print("❸ VERBETERINGEN")
print("="*70)

tips = """
✓ Isolatie: verpak beker
✓ Regelmatig wegen: elke 5 min (niet alleen start/eind)
✓ Thermometer: digitaal (±0.1°C)
✓ Constant vermogen: controleer element
✓ Herhaal: 3x meten
✓ Fasen: bepaal opwarm-, kook- en verdampingsfase
✓ Warmteverlies: apart experiment
"""
print(tips)
print("="*70)